In [7]:
import simpy
import numpy as np
import pandas as pd
import random
import math
RANDOM_SEED = 42
### problem paramters ####
order_time_max = 5
order_time_min = 1
food_prepare_min = 2
food_prepare_max = 6

t_inter = 4
n_counter = 2
n_customer = 20
SIM_TIME = 12 * 60 #12 hours



random.seed(RANDOM_SEED)

In [12]:
# set up monitoring

# class CounterWaitingTime(simpy.Resource):
#     def __init__(self,*args, **kwargs):
#         super().__init__(*args,**kwargs)
#         self.data = []
#     def request(self,*args,**kwargs):
#         self.data.append((self._env.now,len(self.queue)))
#         return  super().request(*args,**kwargs)
#     def release(self, *args,**kwargs):
#         self.data.append((self._env.now,len(self.queue)))
#         return super().release(*args,**kwargs)
# def setup_monitoring(env,res):
#     with res.request() as req:
#         yield req
#         yield env.timeout(1)
# res = CounterWaitingTime()
def round_down(n, decimals=1):
    multiplier = 10 ** decimals
    return abs(math.floor(n*multiplier + 0.5) / multiplier)

counter_total_wait_times = 0

# ,"Clock Time","Interarrival Time","Service Time","Wait Time","Queue Length","Total Time In System"
column_names = ["Arrival Time","Service Start Time","Queue time","Exit Time","Total Wait Time","Total Service Time","Total Time in System"]
data = np.zeros((n_customer,len(column_names)))

In [13]:
def customer_arrivals(env, num_counter):
    """Create new *customer* until the sim time reaches 120. with poisson process"""
    counter = Counter(env,num_counter)
    for i in range(n_customer):
        # simulating Poisson process
        yield env.timeout(random.expovariate(1 / 5))
        env.process(customer(env, i+1, counter))

class Counter(object):
#     Counters to take order
    def __init__(self,env,num_counter):
        self.env = env
        self.counter = simpy.Resource(env,num_counter)
        # res = CounterWaitingTime()
        self.counter_waiting = 0
        self.service_start = None
        # self.counter_total_wait_times =None
        
        
    def take_order(self,cus,env):
        self.service_start = env.now
        print("%s is placing order at counter %.2f" %(cus,self.service_start))
        time_taken_to_place_order = np.random.uniform(order_time_min, order_time_max)
        yield self.env.timeout(time_taken_to_place_order)
        print("Order of %s sent to kitchen at %.2f" %(cus, env.now))
        # yield env.process(tank_truck(env, fuel_pump))

    def receive_order(self,cus,env):
        global counter_total_wait_times
        food_prepare_time = np.random.uniform(food_prepare_min,food_prepare_max)
        yield self.env.timeout(food_prepare_time)
        service_end = env.now
        print("%s collected the food at %.2f" %(cus, service_end))
        counter_total_wait_times = counter_total_wait_times+(service_end-self.service_start)

def customer(env, label, order):
#     the customer process arrive at the restaurant and request counter to take order
    label = label-1
    arrive_time = env.now
    print("%s entering the queue at %.2f"%(label,arrive_time))
#     data[label,0]=label
    data[label,0]= round_down(arrive_time)
    with order.counter.request() as my_turn:
        yield my_turn
        service_start = env.now
        data[label,1] = round_down(service_start)
        queue_time = service_start - arrive_time
        data[label,2]= round_down(queue_time)
        # placing order at counter
        yield env.process(order.take_order(label,env))
        # waiting order at counter
        prepare_food_start = env.now
        # counter is idle now
        yield env.process(order.receive_order(label,env))
        # prepare_food_end = round_down(env.now)
        # counter_total_wait_times += round_down(prepare_food_end - prepare_food_start)
        # receive food from counter
        prepare_food_end = env.now
        data[label,3] = round_down(prepare_food_end)

        kitchen_prepare_duration = (prepare_food_end-prepare_food_start)
        # total wait time
        data[label,4] = round_down(kitchen_prepare_duration+queue_time)
        # total service time
        data[label,5] = round_down(prepare_food_end-service_start)
        # total time in system
        data[label,6] = round_down(prepare_food_end-arrive_time)
        # monitor counter idle time
        



env = simpy.Environment()

In [14]:
env.process(customer_arrivals(env, n_counter))
env.run(until=SIM_TIME)

0 entering the queue at 8.19
0 is placing order at counter 8.19
Order of 0 sent to kitchen at 11.29
1 entering the queue at 14.18
1 is placing order at counter 14.18
0 collected the food at 15.87
2 entering the queue at 16.26
2 is placing order at counter 16.26
3 entering the queue at 17.11
Order of 1 sent to kitchen at 18.85
Order of 2 sent to kitchen at 19.97
2 collected the food at 23.09
3 is placing order at counter 23.09
1 collected the food at 24.46
Order of 3 sent to kitchen at 27.65
3 collected the food at 31.71
4 entering the queue at 32.87
4 is placing order at counter 32.87
5 entering the queue at 34.92
5 is placing order at counter 34.92
6 entering the queue at 35.40
7 entering the queue at 35.91
Order of 4 sent to kitchen at 36.83
Order of 5 sent to kitchen at 36.94
5 collected the food at 39.83
6 is placing order at counter 39.83
4 collected the food at 40.99
7 is placing order at counter 40.99
Order of 7 sent to kitchen at 42.11
Order of 6 sent to kitchen at 43.14
6 coll

In [15]:
np_arr = np.array(data).reshape(n_customer,-1)
labels = [*range(1,n_customer+1)]

# np_arr.shape
df=pd.DataFrame(data = np_arr,index=labels,columns=column_names)
df

,Arrival Time,Service Start Time,Queue time,Exit Time,Total Wait Time,Total Service Time,Total Time in System
1,8.2,8.2,0.0,15.9,4.6,7.7,7.7
2,14.2,14.2,0.0,24.5,5.6,10.3,10.3
3,16.3,16.3,0.0,23.1,3.1,6.8,6.8
4,17.1,23.1,6.0,31.7,10.0,8.6,14.6
5,32.9,32.9,0.0,41.0,4.2,8.1,8.1
6,34.9,34.9,0.0,39.8,2.9,4.9,4.9
7,35.4,39.8,4.4,45.1,6.4,5.3,9.7
8,35.9,41.0,5.1,45.9,8.8,4.9,9.9
9,45.3,45.3,0.0,54.9,5.5,9.6,9.6
10,49.9,49.9,0.0,58.2,4.1,8.2,8.2


In [17]:
total_wait_time = np.sum(df["Total Wait Time"])

total_time_in_system = np.sum(df["Total Time in System"])

total_service_time = np.sum(df['Total Service Time'])


print("")

def display(i):
    print()
    print("Counter Total Idle Time => %.2f Minutes" % counter_total_wait_times)
    print()
    print("Average Waiting Time => %.2f Minutes" % (total_wait_time / i))
    print("Average Service Time => %.2f Minutes" % (total_service_time / i))
    print("Average Time Spent In System => %.2f Minutes" % (total_time_in_system / i))

display(n_customer)



Counter Total Idle Time => 65.95 Minutes

Average Waiting Time => 4.88 Minutes
Average Service Time => 7.12 Minutes
Average Time Spent In System => 7.93 Minutes
